# Import thư viện 

In [ ]:
import requests
import pandas as pd
from pathlib import Path


# Tạo nơi để lưu dữ liệu

In [ ]:
OUT = Path(r"C:\Users\Admin\Desktop\TANPHAT\hocotruong\Năm ba 2025-2026\HK1_A\Thu thập và tiền xử lý dữ liệu\Đồ_án_GDP\data\data_raw")
OUT.mkdir(parents=True, exist_ok=True)

# Lấy dữ liệu từ WorldBank bằng Api

In [ ]:
def crawl_worldbank_raw(indicator: str, country: str = "VN", out_dir: Path = OUT):
    """
    📘 Mô tả:
    Lấy dữ liệu thô (raw) từ World Bank API cho một chỉ số kinh tế cụ thể (indicator).
    - Giữ nguyên giá trị NaN
    - Không ép kiểu dữ liệu
    - Không sắp xếp lại (giữ đúng thứ tự API trả về)
    - Lưu dữ liệu ra CSV
    
    Ví dụ:
        crawl_worldbank_raw("NY.GDP.MKTP.CD", "VN")  # GDP của Việt Nam
    """

    # ----------------------------
    # 1️⃣ Cấu hình API cơ bản
    # ----------------------------
    base = f"https://api.worldbank.org/v2/country/{country}/indicator/{indicator}"  # URL API World Bank
    page, per_page, all_rows = 1, 1000, []  # Trang đầu tiên, mỗi trang tối đa 1000 dòng, danh sách chứa dữ liệu

    # ----------------------------
    # 2️⃣ Lặp qua từng trang (phân trang API)
    # ----------------------------
    while True:
        # Gửi yêu cầu HTTP GET đến API, yêu cầu dữ liệu JSON
        r = requests.get(
            base,
            params={"format": "json", "per_page": per_page, "page": page},
            timeout=30
        )

        # Nếu request lỗi HTTP (404, 500, timeout, ...) → ném lỗi để dừng
        r.raise_for_status()

        # Parse phản hồi JSON từ server
        payload = r.json()

        # Kiểm tra tính hợp lệ của payload
        # World Bank API trả dạng [metadata, data]
        if not isinstance(payload, list) or len(payload) < 2 or payload[1] is None:
            break  # Nếu không hợp lệ hoặc hết dữ liệu → thoát vòng lặp

        # Tách metadata và dữ liệu thật
        meta, data = payload[0], payload[1]

        # Nếu không có dữ liệu (trang trống) → kết thúc
        if not data:
            break

        # ----------------------------
        # 3️⃣ Duyệt từng phần tử (item) trong danh sách dữ liệu
        # ----------------------------
        for item in data:
            all_rows.append({
                "indicator_id":    (item.get("indicator") or {}).get("id"),       # Mã chỉ số (VD: NY.GDP.MKTP.CD)
                "indicator_value": (item.get("indicator") or {}).get("value"),    # Tên chỉ số (VD: GDP (current US$))
                "country_id":      (item.get("country") or {}).get("id"),         # Mã quốc gia (VD: VN)
                "country_value":   (item.get("country") or {}).get("value"),      # Tên quốc gia (VD: Vietnam)
                "countryiso3code": item.get("countryiso3code"),                   # ISO3 code (VD: VNM)
                "date":            item.get("date"),                              # Năm (VD: 2023)
                "value":           item.get("value"),                             # Giá trị của chỉ số (VD: GDP = 409B)
                "unit":            item.get("unit"),                              # Đơn vị (thường None)
                "obs_status":      item.get("obs_status"),                        # Trạng thái quan sát (thường None)
                "decimal":         item.get("decimal"),                           # Số chữ số thập phân
            })

        # ----------------------------
        # 4️⃣ Kiểm tra đã đến trang cuối chưa
        # ----------------------------
        if page >= meta.get("pages", 1):  # Nếu page hiện tại >= tổng số trang
            break                         # Dừng vòng lặp
        page += 1                         # Ngược lại → sang trang kế tiếp

    # ----------------------------
    # 5️⃣ Tạo DataFrame từ danh sách all_rows
    # ----------------------------
    df = pd.DataFrame(all_rows, columns=[
        "indicator_id", "indicator_value",
        "country_id", "country_value",
        "countryiso3code", "date",
        "value", "unit", "obs_status", "decimal"
    ])

    # ----------------------------
    # 6️⃣ Lưu kết quả ra file CSV
    # ----------------------------
    out_path = out_dir / f"{indicator}_{country}_WB_RAW.csv"  # Tên file xuất
    df.to_csv(out_path, index=False, encoding="utf-8-sig")    # Lưu với encoding tránh lỗi font tiếng Việt

    # ----------------------------
    # 7️⃣ In kết quả ra màn hình
    # ----------------------------
    print(f"✅ WorldBank {indicator} ({country}) -> {len(df)} dòng | {out_path}")

    # ----------------------------
    # 8️⃣ Trả về DataFrame để tiếp tục xử lý
    # ----------------------------
    return df


Code chính để crawl dữ liệu từ WorldBank bằng Api

Những dữ liệu crawl chính:

-Inflation (Lạm phát / % )

-CPI index (Chỉ số giá tiêu dùng 2010)

Lưu dữ liệu vào file đã chỉ định 

In [ ]:
 # Inflation (annual %)
wb_inflation_pct = crawl_worldbank_raw("FP.CPI.TOTL.ZG", "VN",OUT) 

In [ ]:
# CPI index (2010=100)
wb_cpi_index     = crawl_worldbank_raw("FP.CPI.TOTL",    "VN",OUT)  